In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/Colab Notebooks/preprocess'
data_path = f'/content/{root_path}/DATA/'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np

In [0]:
df = pd.read_csv(f'/content/{root_path}/DATA/career.csv')

In [6]:
df.head()

,id,title,question_body,tags,answers,scores,url
0,200842,What is the most challenging part of being a S...,#education #teacher,|education|teacher,NaN,0,https://www.careervillage.org/questions/200842...
1,200769,"I have business studies ,coputer application t...",Hi i am lerato and 15 years of age i live in s...,|journalism|business|entrepreneur,NaN,0,https://www.careervillage.org/questions/200769...
2,200723,how do i get my ged,i need ged #general,|general,NaN,0,https://www.careervillage.org/questions/200723...
3,200721,What steps would you recommend I take to ente...,#medicine #healthcare #medical Assistant #hosp...,|medicine|healthcare|medical|hospital-and-heal...,NaN,0,https://www.careervillage.org/questions/200721...
4,200718,What should i know about being a Combat Medic?,I'm a student at Job Corps still trying to fig...,|medicalfield|army,NaN,0,https://www.careervillage.org/questions/200718...


In [7]:
df.shape

(6798, 7)

In [0]:
df.dropna(inplace=True)

In [0]:
df1 = df.copy()

In [0]:
import re
import nltk
import inflect
from nltk.corpus import stopwords

def tokenize_text(text):
    "Apply tokenization using spacy to docstrings."
    tokens = EN.tokenizer(text)
    return [token.text.lower() for token in tokens if not token.is_space]

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def normalize(words):
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = remove_stopwords(words)
    return words

def tokenize_code(text):
    "A very basic procedure for tokenizing code strings."
    return RegexpTokenizer(r'\w+').tokenize(text)

def preprocess_text(text):
    return ' '.join(normalize(tokenize_text(text)))

In [11]:
df1.head(1)

,id,title,question_body,tags,answers,scores,url
5,200717,How difficult is computer networking,I am a student who is a computer nerd somewhat...,|computer|information-technology|computer-netw...,Computer networking defines rules on how a com...,0,https://www.careervillage.org/questions/200717...


In [0]:
from bs4 import BeautifulSoup
from textblob import TextBlob

title_list = [] 
content_list = []
url_list = df1.url.to_list()
comment_list = []
sentiment_polarity_list = []
sentiment_subjectivity_list = []
vote_list =[]
tag_list = []
corpus_list = []

for i, row in df1.iterrows():
    title_list.append(row.title)    # Get question title
    tag_list.append(row.tags)     # Get question tags
    
    # Questions
    content = row.question_body
    soup = BeautifulSoup(content, 'lxml')
    if soup.code: soup.code.decompose()     # Remove the code section
    tag_p = soup.p
    tag_pre = soup.pre
    text = ''
    if tag_p: text = text + tag_p.get_text()
    if tag_pre: text = text + tag_pre.get_text()
        
    content_list.append(str(row.title) + ' ' + str(text))   # Append title and question body data to the updated question body
    
    # url_list.append('https://stackoverflow.com/questions/' + str(row.id.values[0]))
    
    # Answers
    content = row.answers
    soup = BeautifulSoup(content, 'lxml')
    if soup.code: soup.code.decompose()
    tag_p = soup.p
    tag_pre = soup.pre
    text = ''
    if tag_p: text = text + tag_p.get_text()
    if tag_pre: text = text + tag_pre.get_text()
    comment_list.append(text)
    
    vote_list.append(row.scores)       # Append votes
    
    corpus_list.append(content_list[-1] + ' ' + comment_list[-1])     # Combine the updated body and answers to make the corpus
    
    sentiment = TextBlob(row.answers).sentiment
    sentiment_polarity_list.append(sentiment.polarity)
    sentiment_subjectivity_list.append(sentiment.subjectivity)

content_token_df = pd.DataFrame({'original_title': title_list, 'post_corpus': corpus_list, 'question_content': content_list, 'question_url': url_list, 'tags': tag_list, 'overall_scores':vote_list,'answers_content': comment_list, 'sentiment_polarity': sentiment_polarity_list, 'sentiment_subjectivity':sentiment_subjectivity_list})

In [13]:
content_token_df.head(1)

,original_title,post_corpus,question_content,question_url,tags,overall_scores,answers_content,sentiment_polarity,sentiment_subjectivity
0,How difficult is computer networking,How difficult is computer networking I am a st...,How difficult is computer networking I am a st...,https://www.careervillage.org/questions/200717...,|computer|information-technology|computer-netw...,0,Computer networking defines rules on how a com...,-0.218333,0.399762


In [0]:
content_token_df.tags = content_token_df.tags.apply(lambda x: x.split('|'))   # Convert raw text data of tags into lists
content_token_df.tags = content_token_df.tags.apply(lambda x: [i for i in x if i ]) 
# Make a dictionary to count the frequencies for all tags
tag_freq_dict = {}
for tags in content_token_df.tags:
    for tag in tags:
        if tag not in tag_freq_dict:
            tag_freq_dict[tag] = 0
        else:
            tag_freq_dict[tag] += 1

In [15]:
content_token_df.head(1)

,original_title,post_corpus,question_content,question_url,tags,overall_scores,answers_content,sentiment_polarity,sentiment_subjectivity
0,How difficult is computer networking,How difficult is computer networking I am a st...,How difficult is computer networking I am a st...,https://www.careervillage.org/questions/200717...,"[computer, information-technology, computer-ne...",0,Computer networking defines rules on how a com...,-0.218333,0.399762


In [0]:
import heapq
most_common_tags = heapq.nlargest(300, tag_freq_dict, key=tag_freq_dict.get)

In [17]:
most_common_tags

['career',
 'college',
 'medicine',
 'engineering',
 'business',
 'job',
 'healthcare',
 'technology',
 'nursing',
 'doctor',
 'tech',
 'psychology',
 'computer',
 'college-major',
 'engineer',
 'nurse',
 'art',
 'education',
 'money',
 'science',
 'computer-science',
 'career-choice',
 'finance',
 'law',
 'college-advice',
 'job-search',
 'criminal-justice',
 'student',
 'teaching',
 'entrepreneur',
 'design',
 'internship',
 'military',
 'biology',
 'law-enforcement',
 'major',
 'construction',
 'teacher',
 'animals',
 'lawyer',
 'math',
 'career-path',
 'chef',
 'programming',
 'veterinary-medicine',
 'career-counseling',
 'veterinarian',
 'sports',
 'information-technology',
 'computer-software',
 'mechanical-engineering',
 'management',
 'veterinary',
 'graphic-design',
 'marketing',
 'video-games',
 'registered-nurses',
 'culinary-arts',
 'cooking',
 'work',
 'artist',
 'college-admissions',
 'scholarship',
 'welding',
 'school',
 'electrical-engineering',
 'business-management',

In [0]:
final_indices = []
for i,tags in enumerate(content_token_df.tags.values.tolist()):
    if len(set(tags).intersection(set(most_common_tags)))>1:   # The minimum length for common tags should be 2 because 'python' is a common tag for all
        final_indices.append(i)

In [0]:
final_data = content_token_df.iloc[final_indices]

In [20]:
nltk.download("popular")

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

True

In [21]:
import spacy
EN = spacy.load('en_core_web_sm')

# Preprocess text for 'question_body', 'post_corpus' and a new column 'processed_title'
final_data.question_content = final_data.question_content.apply(lambda x: preprocess_text(x))
final_data.post_corpus = final_data.post_corpus.apply(lambda x: preprocess_text(x))
final_data['processed_title'] = final_data.original_title.apply(lambda x: preprocess_text(x))

# Normalize numeric data for the scores
# final_data.overall_scores = (final_data.overall_scores - final_data.overall_scores.mean()) / (final_data.overall_scores.max() - final_data.overall_scores.min())

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [22]:
final_data.tags = final_data.tags.apply(lambda x: '|'.join(x))    # Combine the lists back into text data
final_data.drop(['answers_content'], axis=1,inplace=True)    # Remove the answers_content columns because it is alreaady included in the corpus
final_data.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,original_title,post_corpus,question_content,question_url,tags,overall_scores,sentiment_polarity,sentiment_subjectivity,processed_title
0,How difficult is computer networking,difficult computer networking student computer...,difficult computer networking student computer...,https://www.careervillage.org/questions/200717...,computer|information-technology|computer-netwo...,0,-0.218333,0.399762,difficult computer networking
3,How would I go about becoming a welder,would go becoming welder like hands stuff acti...,would go becoming welder like hands stuff acti...,https://www.careervillage.org/questions/200389...,welding|welder,2,-0.028231,0.390816,would go becoming welder
5,How hard is it to complete the years of school...,hard complete years school residency orthopedi...,hard complete years school residency orthopedi...,https://www.careervillage.org/questions/200577...,surgeon|orthopedicsurgeon|orthopedic|medical|m...,2,0.079545,0.549242,hard complete years school residency orthopedi...
6,Should I take physics or AP Bio senior year?,take physics ap bio senior year guidance couns...,take physics ap bio senior year guidance couns...,https://www.careervillage.org/questions/200525...,college|biology|physics|college-major|science|...,2,0.197667,0.464855,take physics ap bio senior year
7,What is the average salary of an actor?,average salary actor theatre acting actor non ...,average salary actor theatre acting actor,https://www.careervillage.org/questions/200445...,theatre|acting|actor,4,-0.025242,0.489881,average salary actor


In [0]:
final_data.to_csv(f'{data_path}Preprocessed_data.csv', index=False)